# Regulární výrazy

Součástí Pythonu je standardní modul `re` pro práci s regulárními výrazy, ale existuje též přídatná knihovna `regex`, která je rychlejší a má bohatší podporu matchování roztodivných vlastností znaků definovaných Unicodem. Tu je potřeba nainstalovat zvlášť. Pokud je k dispozici (na Jupyteru je), není důvod jí nedat přednost.

Při importu se knihovna `regex` často přejmenovává na prostorově úspornější `re` (viz další buňka). [API](https://en.wikipedia.org/wiki/Application_programming_interface) obou knihoven je vzájemně kompatibilní (poskytují uživateli stejně pojmenované funkce se stejně pojmenovanými argumenty, které se chovají totožně), takže pokud program přenesete na počítač, kde knihovna `regex` není k dispozici, stačí jen přepsat `import regex as re` na `import re` a program půjde spustit. V praxi je ale ještě potřeba ověřit, že vaše regulární výrazy nevyužívají pokročilou podporu Unicodu, která je dostupná jen v `regex`, a vrací i s knihovnou `re` očekávané výsledky.

In [ ]:
import regex as re

## Rychlý přehled

Přehled syntaxe regulárních výrazů podle [NLTK Book](http://www.nltk.org/book/ch03.html). Slouží jen k rychlé orientaci, teď jej přeskočte, věnujte se raději interaktivním příkladům níže a vracejte se k němu jen podle potřeby (např. kvůli objasnění účelu různých speciálních operátorů).

```
.         # Wildcard, matches any character
\w   \W   # Matches any (non-)word character (careful, the
          # computer's idea about what a word character is might
          # be different from yours)
\d   \D   # Matches any (non-)digit character
\s   \S   # Matches any (non-)space character
\p{...}   # Matches any character with Unicode property ...
\P{...}   # Matches any character without Unicode property ...
^abc      # Matches some pattern abc at the start of a string
          # (or line, if the multiline flag is enabled)
abc$      # Matches some pattern abc at the end of a string
          # (or line, if the multiline flag is enabled)
\babc\b   # Matches some pattern abc surrounded by word boundaries
\Babc\B   # Matches some pattern abc not surrounded by word boundaries
[abc]     # Matches one of a set of characters
[^abc]    # Matches any character which is NOT in the set
[A-Z0-9]  # Matches one of a range of characters
ed|ing|s  # Matches one of the specified strings (disjunction)
*         # Zero or more of previous item, e.g. a*, [a-z]* (also
          # known as Kleene Closure); greedy (match as many as
          # possible)
*?        # The same as *, but non-greedy (match as few as possible)
+         # One or more of previous item, e.g. a+, [a-z]+; greedy
+?        # The same as + but non-greedy
?         # Zero or one of the previous item (i.e. optional), e.g.
          # a?, [a-z]?
{n}       # Exactly n repeats where n is a non-negative integer
{n,}      # At least n repeats
{,n}      # No more than n repeats
{m,n}     # At least m and no more than n repeats
a(b|c)+   # Parentheses indicate the scope of the operators and
          # capture the corresponding groups of characters, which
          # are then accessible accessible with the match.group()
          # or match.groups() method, or with a backreference:
          # \1, \2 etc., depending on the order of the groups
a(?:b|c)+ # Non-capturing version of the parentheses
```

## Interaktivní cvičení

In [ ]:
# zde si nadefinujeme funkci, která vytvoří interaktivní widget
# pro testování regulárních výrazů; na požádání rád objasním,
# jak funguje, ale není účelem tomuhle kódu rozumět
import IPython.core.display as ipd
import ipywidgets as ipw

def findall(dotall=False, multiline=False, ignorecase=False, only_first=False, regex="", string=""):
    flags = 0
    if dotall:
        flags |= re.DOTALL
    if multiline:
        flags |= re.MULTILINE
    if ignorecase:
        flags |= re.IGNORECASE
    start = '<span style="background-color: gold">'
    end = "</span>"
    offset_bump = len(start) + len(end)
    offset = 0
    html = string
    matches = []
    if regex:
        try:
            for m in re.finditer(regex, string, flags):
                matches.append(m.captures()[0])
                span = m.span()
                sstart, send = span[0] + offset, span[1] + offset
                html = html[:sstart] + start + html[sstart:send] + end + html[send:]
                offset += offset_bump
                if only_first:
                    break
        except:
            pass
    ipd.display(ipd.HTML("<p>REGEX: <strong>" + regex + "</strong></p><p><pre>" + html + "</pre></p>"))
    return matches

def interactive_findall(string):
    ipw.interact(findall, string=ipw.fixed(string))

In [ ]:
# nadefinujeme si pár textových řetězců na hraní s regulárními výrazy

MARY = """Mary had a little lamb.
And everywhere that Mary
went, the lamb was sure
to go."""

SPECIAL = "Special characters must be escaped.*"

PETS = "The pet store sold cats, dogs, and birds."

FIRST = "=first first= # =second second= # =first= # =second="

QUANT1 = """Match with zero in the middle: @@
Subexpresion occurs, but...: @=#=ABC@
Lots of occurrences: @=#==#==#==#==#=@
Must repeat entire pattern: @=#==#=#==#=@"""

QUANT2 = """AAAD
ABBBBCD
BBBCD
ABCCD
AAABBBC"""

QUANT3 = """aaaaa bbbbb ccccc
aaa bbb ccc
aaaaa bbbbbbbbbbbbbb ccccc"""

BACK = """jkl abc xyz
jkl xyz abc
jkl abc abc
jkl xyz xyz"""

LAZY = """-- I want to match the words that start
-- with 'th' and end with 's'.
this
thus
thistle
this line matches too much
"""

In [ ]:
# vyhodnoťte a do políčka regex zkuste zadat třeba regulární výraz:
#   .a
# a pak:
#   [a-z]a
interactive_findall(MARY)

In [ ]:
# vyhodnoťte a do políčka regex zkuste zadat třeba regulární výraz:
#   .*
# a potom:
#   \.\*
interactive_findall(SPECIAL)

In [ ]:
# vyhodnoťte a do políčka regex zkuste zadat třeba regulární výraz:
#   cat|dog|bird
interactive_findall(PETS)

In [ ]:
# vyhodnoťte a do políčka regex zkuste zadat třeba regulární výraz:
#   =first|second=
# a potom:
#   =(first|second)=
interactive_findall(FIRST)

In [ ]:
# vyhodnoťte a do políčka regex zkuste zadat třeba regulární výraz:
#   @(=#=)*@
interactive_findall(QUANT1)

In [ ]:
# vyhodnoťte a do políčka regex zkuste zadat třeba regulární výraz:
#   A+B*C?D
interactive_findall(QUANT2)

In [ ]:
# vyhodnoťte a do políčka regex zkuste zadat třeba regulární výraz:
#   a{,4}
interactive_findall(QUANT3)

In [ ]:
# vyhodnoťte a do políčka regex zkuste zadat třeba regulární výraz:
#   (abc|xyz) \1
interactive_findall(BACK)

In [ ]:
# vyhodnoťte a do políčka regex zkuste zadat třeba regulární výraz:
#   \bth\p{Alphabetic}*s\b
# \p{Alphabetic} je nejspolehlivější způsob, jak namatchovat jakýkoli
# "písmenkoidní" znak v co nejširším slova smyslu. mezi složené závorky
# v \p{...} lze napsat název jakékoli kategorie znaků, která je
# definovaná v rámci tzv. Unicode properties (jedna z užitečných je
# např. \p{Punctuation}, jinak Google!)
interactive_findall(LAZY)

## Použití v Pythonu

Zaprvé: regulární výrazy se zapisují jako běžné textové řetězce; jejich speciální chování při hledání vzorců v textu vychází čistě z toho, že tento řetězec pak předáte funkci z modulu `regex` (nebo `re`), která toto chování implementuje:

In [ ]:
re.findall("s.", "My father likes cars.")

Vzhledem k tomu, že regulární výrazy často obsahují speciální sekvence znaků se zpětnými lomítky, je dobré používat pro jejich definici tzv. **raw strings**, tj. `r"..."` místo jen `"..."`, které zajistí, že vám zpětná lomítka nezmizí. Některé sekvence se zpětnými lomítky v řetězcích totiž sám Pythonu považuje za speciální a nahrazuje je jinými znaky:

In [ ]:
print("a\bm")

My ale potřebujeme, aby tam ta zpětná lomítka zůstala, aby je mohla využít příslušná funkce modulu `regex` (nebo `re`), která teprve řetězec jakožto regulární výraz interpretuje. Proto je nejjednodušší všechny regulární výrazy zadávat jako **raw string**:

In [ ]:
print(r"a\bm")

Funkce `re.search()` otestuje, zda regulární výraz matchuje kdekoli v rámci daného řetězce, a pokud ano, vrátí objekt typu `Match`:

In [ ]:
m = re.search(r"s.", "My father likes cars.")
m

Všimněte si, že tak dostaneme pouze první výskyt vzorce odpovídající regulárnímu výrazu. Z objektu `Match` pak můžeme např. vytáhnout obsah matche...

In [ ]:
m.group()

... nebo indexy začátku a konce matche v rámci řetězce:

In [ ]:
m.span()

In [ ]:
"My father likes cars."[14:16]

`re.match` funguje podobně, ale začátek matche musí odpovídat začátku řetězce:

In [ ]:
re.match(r"s.", "My father likes cars.")

In [ ]:
re.match(r"s.", "summertime")

`re.fullmatch()` pak vyžaduje, aby regulární výraz matchoval celý řetězec od začátku do konce:

In [ ]:
re.fullmatch(r"s.", "summertime")

In [ ]:
re.fullmatch(r"s.", "su")

Pokud chcete najít **všechny výskyty** sekvencí odpovídajícíh regulárnímu výrazu, můžete použít buď funkci `re.findall()`, která vrátí seznam všech namatchovaných podřetězců...

In [ ]:
re.findall(r"s.", "My father likes cars.")

... nebo funkci `re.finditer()`, která vrátí iterátor (potenciální kolekci) objektů typu `Match` (tento iterátor sám o sobě je objekt typu `Scanner`),...

In [ ]:
re.finditer(r"s.", "My father likes cars.")

... z něhož je pak pochopitelně potřeba jednotlivé matche postupně vytáhnout (výhoda je, že matche v žádné chvíli nedržíte všechny najednou v paměti, což je šetrnější přístup, než je všechny uložit do seznamu, jako to dělá funkce `re.findall()`, zvlášť pokud je matchů potenciálně opravdu hodně):

In [ ]:
for match in re.finditer(r"s.", "My father likes cars."):
    print(match)
    print(match.span())
    print(repr(match.group()))

Další výhoda `re.finditer()` je v tom, že někdy prostě potřebujete pracovat s celým objektem typu `Match` -- např. potřebujete informaci o tom, kde match v rámci původního řetězce začíná a končí, kterou získáte díky metodě `match.span()`.

**Výstup všech těchto funkcí lze pak používat v `if` podmínkách** a jiných logických operacích, protože při úspěchu vracejí hodnoty (objekty typu `Match` nebo seznamy), které Python považuje za pravdivé (`True`), a při neúspěchu nevracejí nic (resp. speciální hodnotu `None`), kterou naopak Python považuje za nepravdivou (`False`):

In [ ]:
if re.match(r"as", "asdf"):
    print("úspěch!")
else:
    print("smůla :(")

In [ ]:
if re.fullmatch(r"as", "asdf"):
    print("úspěch!")
else:
    print("smůla :(")

Pak je zde ještě funkce `re.sub()`, která umožňuje nahrazovat namatchované části řetězce něčím jiným:

In [ ]:
re.sub(r"cat|like", r"dog", "I like cats and categories.")

Pokud chcete v nahrazovacím řetězci odkázat na obsah či část obsahu matche, je potřeba příslušnou část regulárního výrazu obalit do **skupiny** pomocí závorek `(...)` a pak na skupiny podle jejího pořadí odkázat:

In [ ]:
re.sub(r"(cat|like)", r"dog\1", "I like cats and categories.")

In [ ]:
re.sub(r"(like.*?) and (categories)", r"really \1 but I hate \2", "I like cats and categories.")

In [ ]:
re.sub(r"(cat)|(like)", r"\1dog\2", "I like cats and categories.")

**Chování funkcí lze měnit tzv. přepínači**, které se zadávají pomocí nepovinného parametru `flags`. Existují následující přepínače: `re.ASCII`, `re.DEBUG`, `re.IGNORECASE`, `re.LOCALE`, `re.MULTILINE`, `re.DOTALL`, `re.VERBOSE`. Blíže popsané jsou [v dokumentaci](https://docs.python.org/3/library/re.html#module-contents).

A používají se takto:

In [ ]:
re.match("as", "ASDF", flags=re.IGNORECASE)

Pokud jich chcete použít víc najednou, je potřeba je zřetězit pomocí operátoru `|` (což vypadá obskurně a z hlediska pohodlí uživatele to není moc dobré řešení, ale bohužel je to tak dané; koho by zajímalo, co operátor `|` přesně dělá, Google i já jsme k dispozici, ale není to nijak životně důležité).

In [ ]:
re.match("a.", "A\nSDF", flags=re.IGNORECASE | re.DOTALL)

## Na závěr možná nečekané praktické uplatnění...

U běžných "písmenek", která existují v rámci Unicodu jako jeden znak (tj. i česká písmenka s diakritikou a mnohem exotičtější záležitosti), odpovídá délka řetězce v Pythonu intuitivnímu počtu "písmenek" (**grafémů**), kdybychom je počítali ručně:

In [ ]:
len("ručně")

Někdy jsou ovšem grafémy složené z více znaků (jeden základní + další diakritické), typicky tomu tak bývá u fonetických přepisů, ale může se tak stát i jinde:

In [ ]:
list("tr̝̥i")

In [ ]:
len("tr̝̥i")

Tam už pak počet znaků intuitivnímu / vizuálně určenému počtu grafémů neodpovídá. Pokud chceme grafémy **spočítat spolehlivě automaticky**, musíme použít regulární výraz `\X`, který matchuje znak či sekvenci znaků, která odpovídá právě jednomu vizuálnímu grafému:

In [ ]:
re.findall(r"\X", "tr̝̥i")

In [ ]:
len(re.findall(r"\X", "tr̝̥i"))

In [ ]:
# nebo též
sum(1 for _ in re.finditer(r"\X", "tr̝̥i"))

Regulární výraz `\X` je právě jedna z pokročilých Unicodových funkcí, která **není dostupná ve standardním modulu `re`**.